In [1]:
import glob, os
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack
import matplotlib as mpl
from astropy import units as u
import speclite.filters
import time
import sncosmo

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 200
plt.rc('font', family='serif')

In [4]:
from scipy.optimize import curve_fit
def func(x, a):
	return a*x
def calc_chisquare(obs, exp):
	return np.sum((obs-exp)**2/exp)

In [8]:
from helper import get_speclite_med
meds = get_speclite_med()
mlam = meds.effective_wavelengths

In [6]:
intbl = ascii.read(f"../5.result/kn_sim_cube_obs/7dt_med_180s_iter1_40Mpc/Run_TP_dyn_all_lanth_wind1_all_md0.001_vd0.05_mw0.001_vw0.05_angle0_synphot_med_obs_d40_iter1.7dtmed.ecsv")
intbl[:3]

seed,t,magabs_m400,magapp_m400,snr_m400,magerr_m400,magobs_m400,fnu_m400,fnuobs_m400,fnuerr_m400,detection_m400,magabs_m425,magapp_m425,snr_m425,magerr_m425,magobs_m425,fnu_m425,fnuobs_m425,fnuerr_m425,detection_m425,magabs_m450,magapp_m450,snr_m450,magerr_m450,magobs_m450,fnu_m450,fnuobs_m450,fnuerr_m450,detection_m450,magabs_m475,magapp_m475,snr_m475,magerr_m475,magobs_m475,fnu_m475,fnuobs_m475,fnuerr_m475,detection_m475,magabs_m500,magapp_m500,snr_m500,magerr_m500,magobs_m500,fnu_m500,fnuobs_m500,fnuerr_m500,detection_m500,magabs_m525,magapp_m525,snr_m525,magerr_m525,magobs_m525,fnu_m525,fnuobs_m525,fnuerr_m525,detection_m525,magabs_m550,magapp_m550,snr_m550,magerr_m550,magobs_m550,fnu_m550,fnuobs_m550,fnuerr_m550,detection_m550,magabs_m575,magapp_m575,snr_m575,magerr_m575,magobs_m575,fnu_m575,fnuobs_m575,fnuerr_m575,detection_m575,magabs_m600,magapp_m600,snr_m600,magerr_m600,magobs_m600,fnu_m600,fnuobs_m600,fnuerr_m600,detection_m600,magabs_m625,magapp_m625,snr_m625,magerr_m625,magobs_m625,fnu_m625,fnuobs_m625,fnuerr_m625,detection_m625,magabs_m650,magapp_m650,snr_m650,magerr_m650,magobs_m650,fnu_m650,fnuobs_m650,fnuerr_m650,detection_m650,magabs_m675,magapp_m675,snr_m675,magerr_m675,magobs_m675,fnu_m675,fnuobs_m675,fnuerr_m675,detection_m675,magabs_m700,magapp_m700,snr_m700,magerr_m700,magobs_m700,fnu_m700,fnuobs_m700,fnuerr_m700,detection_m700,magabs_m725,magapp_m725,snr_m725,magerr_m725,magobs_m725,fnu_m725,fnuobs_m725,fnuerr_m725,detection_m725,magabs_m750,magapp_m750,snr_m750,magerr_m750,magobs_m750,fnu_m750,fnuobs_m750,fnuerr_m750,detection_m750,magabs_m775,magapp_m775,snr_m775,magerr_m775,magobs_m775,fnu_m775,fnuobs_m775,fnuerr_m775,detection_m775,magabs_m800,magapp_m800,snr_m800,magerr_m800,magobs_m800,fnu_m800,fnuobs_m800,fnuerr_m800,detection_m800,magabs_m825,magapp_m825,snr_m825,magerr_m825,magobs_m825,fnu_m825,fnuobs_m825,fnuerr_m825,detection_m825,magabs_m850,magapp_m850,snr_m850,magerr_m850,magobs_m850,fnu_m850,fnuobs_m850,fnuerr_m850,detection_m850,magabs_m875,magapp_m875,snr_m875,magerr_m875,magobs_m875,fnu_m875,fnuobs_m875,fnuerr_m875,detection_m875
,d,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),uJy,uJy,uJy,
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,f

In [17]:
t = 0.125

mfilterlist = [f"m{lam}" for lam in np.arange(400, 875+25, 25)]
yobs = np.array([intbl[f"fnuobs_{filte}"][(intbl['t']==t)].item() for filte in mfilterlist])
yobserr = np.array([intbl[f"fnuerr_{filte}"][(intbl['t']==t)].item() for filte in mfilterlist])

In [50]:
t_sn = 0.0

sntbl = ascii.read("../3.table/sn_sim_sncosmo_synphot/hsiao_v3.0_type_Ia_sn_z0.0001_synphot_med.ecsv")

xsim = np.array([sntbl[filte][sntbl['t']==t_sn].to(u.uJy).value.item() for filte in mfilterlist])


In [83]:
niter = int(1e4)
deltlist = []
for ii in range(niter):
	t0 = time.time()
	popt, pcov = curve_fit(
		f=func,
		xdata=xsim,
		ydata=yobs,
		sigma=yobserr,
		absolute_sigma=True,
		check_finite=True,
	)
	delt = time.time()-t0
	# print(f"{delt:1.10f} sec")
	deltlist.append(delt)

print(f"{np.median(deltlist)*1e3:1.3f} +/- {np.std(deltlist)*1e3:1.3f}ms")

0.068 +/- 0.014ms


In [84]:
p0 = np.median(yobs)/np.median(xsim)
p0

3.8588221806011944e-08

In [87]:
deltlist = []
for ii in range(niter):

	t0 = time.time()
	popt, pcov = curve_fit(
		f=func,
		xdata=xsim,
		ydata=yobs,
		p0=p0,
		sigma=yobserr,
		absolute_sigma=True,
		check_finite=True,
	)
	delt = time.time()-t0
	# print(f"{delt:1.10f} sec")
	deltlist.append(delt)

print(f"{np.median(deltlist)*1e3:1.3f} +/- {np.std(deltlist)*1e3:1.3f}ms")

0.050 +/- 0.018ms


In [82]:
popt

array([3.81250886e-08])

In [81]:
f"x{0.069/0.049:1.3f}"

'x1.408'